In [1]:
import pandas as pd

from pyspark.sql.types import DoubleType, ArrayType, StructType, StructField, StringType, IntegerType, FloatType

from pyspark.sql import SparkSession
from pyspark.sql.functions import udf,from_unixtime, min, max, to_date, pandas_udf, col, PandasUDFType, lit, round
from pyspark.sql.types import DoubleType, ArrayType, StructType, StructField, StringType, IntegerType, FloatType
from pyspark.sql import functions as F
from pyspark.sql import Window

import os, time
import subprocess
import os,shutil
from datetime import datetime
import pandas as pd
import numpy as np
from IPython.display import display, HTML

import requests
from shapely.geometry import LineString, Polygon
from shapely.ops import transform
import pyproj
from functools import partial
from shapely.geometry import LineString
from shapely.ops import transform
from pyproj import Proj, Transformer
import pandas as pd
import folium
from shapely.geometry import Polygon
from shapely.ops import unary_union
import shapely.geometry
import h3

from sklearn.preprocessing import LabelEncoder

# Settings
project = "project_aiu"


# Getting today's date
today = datetime.today().strftime('%d %B %Y')

# Spark Session Initialization
shutil.copy("/runtime-addons/cmladdon-2.0.40-b150/log4j.properties", "/etc/spark/conf/") # Setting logging properties

spark = SparkSession.builder \
    .appName("OSN ADEP ADES Identification") \
    .config("spark.log.level", "ERROR")\
    .config("spark.hadoop.fs.azure.ext.cab.required.group", "eur-app-aiu-dev") \
    .config("spark.kerberos.access.hadoopFileSystems", "abfs://storage-fs@cdpdldev0.dfs.core.windows.net/data/project/aiu.db/unmanaged") \
    .config("spark.driver.cores", "1") \
    .config("spark.driver.memory", "8G") \
    .config("spark.executor.memory", "5G") \
    .config("spark.executor.cores", "1") \
    .config("spark.executor.instances", "2") \
    .config("spark.dynamicAllocation.maxExecutors", "6") \
    .config("spark.network.timeout", "800s") \
    .config("spark.executor.heartbeatInterval", "400s") \
    .enableHiveSupport() \
    .getOrCreate()

# Get environment variables
engine_id = os.getenv('CDSW_ENGINE_ID')
domain = os.getenv('CDSW_DOMAIN')

# Format the URL
url = f"https://spark-{engine_id}.{domain}"

# Display the clickable URL
display(HTML(f'<a href="{url}">{url}</a>'))

airports_df = spark.sql(f"""
    SELECT id, ident, iso_country, continent, latitude_deg, longitude_deg, elevation_ft, type
    FROM {project}.oa_airports
    WHERE (type = 'large_airport' OR type = 'medium_airport');
""")

import math
import json

def generate_circle_polygon(lon, lat, radius_nautical_miles, num_points=360):
    """
    Generate a polygon in GeoJSON format around a given latitude and longitude
    with a specified radius in nautical miles.
    
    :param lat: Latitude of the center point
    :param lon: Longitude of the center point
    :param radius_nautical_miles: Radius in nautical miles
    :param num_points: Number of points to generate for the polygon
    :return: A dictionary representing the polygon in GeoJSON format
    """
    # Convert radius from nautical miles to kilometers
    radius_km = radius_nautical_miles * 1.852
    
    # Function to convert from degrees to radians
    def degrees_to_radians(degrees):
        return degrees * math.pi / 180
    
    # Function to calculate the next point given a distance and bearing
    def calculate_point(lon, lat, distance_km, bearing):
        R = 6371.01  # Earth's radius in kilometers
        lat_rad = degrees_to_radians(lat)
        lon_rad = degrees_to_radians(lon)
        distance_rad = distance_km / R
        bearing_rad = degrees_to_radians(bearing)
        
        lat_new_rad = math.asin(math.sin(lat_rad) * math.cos(distance_rad) +
                                math.cos(lat_rad) * math.sin(distance_rad) * math.cos(bearing_rad))
        lon_new_rad = lon_rad + math.atan2(math.sin(bearing_rad) * math.sin(distance_rad) * math.cos(lat_rad),
                                           math.cos(distance_rad) - math.sin(lat_rad) * math.sin(lat_new_rad))
                                           
        lat_new = math.degrees(lat_new_rad)
        lon_new = math.degrees(lon_new_rad)
        return [lon_new, lat_new]
    
    # Generate points
    points = []
    for i in range(num_points):
        bearing = 360 / num_points * i
        point = calculate_point(lon, lat, radius_km, bearing)
        points.append(point)
    points.append(points[0])  # Close the polygon by repeating the first point
    
    # Create GeoJSON
    geojson = {
        "type": "Polygon",
        "coordinates": [points]
    }
    
    geojson_str = json.dumps(geojson)
    
    return geojson_str

def fill_polygon_with_hexagons(polygon_json, resolution=8):
    """Fills a polygon defined by the given JSON with hexagons with defined resolution.

    Args:
        polygon_json (str): A JSON string defining the polygon.
        resolution (int): The H3 resolution for the hexagons.

    Returns:
        list: The list contains the compact hexagon IDs.
    """
    polygon = json.loads(polygon_json)
    hexagons = h3.polyfill(polygon, resolution, geo_json_conformant=True)
    return hexagons

def fill_polygon_with_compact_hexagons(polygon_json, resolution=8):
    """Fills a polygon defined by the given JSON with compact hexagons.

    Args:
        polygon_json (str): A JSON string defining the polygon.
        resolution (int): The H3 resolution for the hexagons.

    Returns:
        tuple: A tuple containing two lists. The first list contains the compact hexagon IDs,
               and the second list contains the resolutions of these hexagons.
    """
    hexagons = fill_polygon_with_hexagons(polygon_json, resolution)
    compact_hexagons = list(h3.compact(hexagons))
    compact_hexagons_resolutions = [h3.get_resolution(h) for h in compact_hexagons]
    return compact_hexagons, compact_hexagons_resolutions

generate_circle_polygon_udf = udf(generate_circle_polygon, StringType())

compact_hex_result_type = StructType([
    StructField("compact_hexagons", ArrayType(StringType()), False),
    StructField("compact_hexagons_resolutions", ArrayType(IntegerType()), False)
])

fill_polygon_with_compact_hexagons_udf = udf(fill_polygon_with_compact_hexagons, compact_hex_result_type)


Setting spark.hadoop.yarn.resourcemanager.principal to quinten.goens


Hive Session ID = 23e3ba3f-64c1-4e2c-9dad-3c332d3f69ca


In [2]:
num_points = 360
max_resolution = 13
radia_nm = [0, 5, 10, 15, 20, 30, 40, 50, 60, 70, 80, 90, 100]

df = pd.DataFrame.from_dict({ #c stands for circle
    'max_resolution' : max_resolution,
    'number_of_points_c' : num_points, 
    'min_c_radius_nm' : radia_nm
})
df['max_c_radius_nm'] = df['min_c_radius_nm'].shift(-1).fillna(np.max(radia_nm)+10)

df['min_c_radius_nm'] = df['min_c_radius_nm'].astype(float)
df['max_c_radius_nm'] = df['max_c_radius_nm'].astype(float)
df['m_col'] = 1

# Define the schema corresponding to your Pandas DataFrame structure
schema = StructType([
    StructField("max_resolution", IntegerType(), True),
    StructField("number_of_points_c", IntegerType(), True),
    StructField("min_c_radius_nm", FloatType(), True),
    StructField("max_c_radius_nm", FloatType(), True),
    StructField("m_col", IntegerType(), True)
])

# Create PySpark DataFrame from Pandas DataFrame using the defined schema
sdf = spark.createDataFrame(df, schema=schema)

In [3]:

airports_df = airports_df.withColumn('m_col', lit(1)).join(sdf, on = 'm_col', how = 'left')

In [4]:
def combine_polygons(outer_geojson_str, inner_geojson_str):
    """
    Combine two GeoJSON polygons into a single "donut-shaped" polygon.
    
    :param outer_geojson_str: GeoJSON string of the outer polygon
    :param inner_geojson_str: GeoJSON string of the inner polygon
    :return: GeoJSON string of the combined polygon with the inner area as a hole
    """
    # Parse the GeoJSON strings into Python dictionaries
    outer_geojson = json.loads(outer_geojson_str)
    inner_geojson = json.loads(inner_geojson_str)
    
    # Extract the coordinates from the inner polygon
    inner_coordinates = inner_geojson['coordinates']
    
    # Insert the inner coordinates as a hole in the outer polygon
    combined_coordinates = outer_geojson['coordinates'] + [inner_coordinates[0]]
    
    # Create a new GeoJSON object for the combined polygon
    combined_geojson = {
        "type": "Polygon",
        "coordinates": combined_coordinates
    }
    
    # Convert the dictionary back to a JSON string
    return json.dumps(combined_geojson)

# Register the UDF
combine_polygons_udf = F.udf(combine_polygons, StringType())

res = airports_df.withColumn(
    "inner_circle_polygon",
    generate_circle_polygon_udf(
        airports_df.longitude_deg,
        airports_df.latitude_deg,
        airports_df.min_c_radius_nm, 
        airports_df.number_of_points_c
    )).withColumn(
    "outer_circle_polygon",
    generate_circle_polygon_udf(
        airports_df.longitude_deg,
        airports_df.latitude_deg,
        airports_df.max_c_radius_nm, 
        airports_df.number_of_points_c
    )).withColumn(
    "combined_circle_polygon",
    combine_polygons_udf(
        F.col("outer_circle_polygon"),
        F.col("inner_circle_polygon")
    )).withColumn(
        "hexagon_data",
        fill_polygon_with_compact_hexagons_udf(
            "combined_circle_polygon",
            "max_resolution" # Resolution
        )
    ).withColumn(
        "hex_id",
        F.col("hexagon_data.compact_hexagons")
    ).withColumn(
        "hex_resolution",
        F.col("hexagon_data.compact_hexagons_resolutions")
    ).toPandas()
    

24/04/22 15:57:39 414 ERROR TaskSetManager: Task 1 in stage 2.0 failed 4 times; aborting job


Py4JJavaError: An error occurred while calling o252.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 2.0 failed 4 times, most recent failure: Lost task 1.3 in stage 2.0 (TID 11) (10.244.13.56 executor 3): java.net.SocketException: Connection reset
	at java.net.SocketOutputStream.socketWrite(SocketOutputStream.java:115)
	at java.net.SocketOutputStream.write(SocketOutputStream.java:155)
	at java.io.BufferedOutputStream.write(BufferedOutputStream.java:122)
	at java.io.DataOutputStream.write(DataOutputStream.java:107)
	at java.io.FilterOutputStream.write(FilterOutputStream.java:97)
	at org.apache.spark.api.python.PythonRDD$.write$1(PythonRDD.scala:295)
	at org.apache.spark.api.python.PythonRDD$.$anonfun$writeIteratorToStream$1(PythonRDD.scala:307)
	at org.apache.spark.api.python.PythonRDD$.$anonfun$writeIteratorToStream$1$adapted(PythonRDD.scala:307)
	at scala.collection.Iterator.foreach(Iterator.scala:941)
	at scala.collection.Iterator.

In [ ]:
res

In [ ]:


pdfs = []
failed_apts = []
for radius_nm in radia_nm:
    print(f"Radius: {radius_nm}")
    for resolution in [13]:
        pdfs = []
        failed_apts = []
        print('Resolution:', resolution)
        for airport in airports_df.toPandas()['ident'].to_list():
            try:
                print('Processing airport: ', airport)
                airports_pdf = airports_df.filter(airports_df.ident == airport).withColumn(
                    "circle_polygon",
                    generate_circle_polygon_udf(
                        airports_df.longitude_deg,
                        airports_df.latitude_deg,
                        lit(radius_nm), 
                        lit(num_points)
                    )
                ).withColumn(
                    "hex_id",
                    fill_circle_with_compact_hexagons_udf(
                        "circle_polygon",
                        lit(resolution) # Resolution
                    )
                ).toPandas()

                airports_pdf = airports_pdf.explode('hex_id')

                airports_pdf['hex_lat'], airports_pdf['hex_lon'] = zip(*airports_pdf['hex_id'].apply(lambda l: h3.h3_to_geo(l)))

                pdfs.append(airports_pdf)
            except Exception as e:
                failed_apts.append(airport)
                print('Airport FAILED:', airport)
                print(f'Error: {e}')

        print(f"Failed airports: {failed_apts}")

        df = pd.concat(pdfs).explode('hex_id')[['id', 'ident', 'latitude_deg', 'longitude_deg', 'elevation_ft', 'type','hex_id', 'hex_lat', 'hex_lon']]

        df.to_parquet(f'data/airport_hex/airport_hex_res_{resolution}_radius_{radius_nm}_nm.parquet')

In [ ]:
pd.read_parquet('data/airport_hex/')